In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show, reset_output
from bokeh.models import HoverTool, ColumnDataSource, CDSView, BooleanFilter, Legend
from bokeh.palettes import Spectral
from bokeh.plotting import figure
from bokeh.models import NumeralTickFormatter
pd.set_option('display.float_format', lambda x:'%.2f'%x)



In [2]:
df_f = pd.read_csv('final_version.csv', parse_dates= True)

df_f.head()

,country_or_area,ISO,commodity_transaction,year,category,new_unit,new_quantity
0,Afghanistan,AFG,Electricity - Gross production,1994,total_electricity,TWh,0.69
1,Afghanistan,AFG,Electricity - Gross production,1995,total_electricity,TWh,0.68
2,Afghanistan,AFG,Electricity - Gross production,1996,total_electricity,TWh,0.68
3,Afghanistan,AFG,Electricity - Gross production,1997,total_electricity,TWh,0.67
4,Afghanistan,AFG,Electricity - Gross production,1998,total_electricity,TWh,0.67


In [3]:
df_r = pd.read_csv('renewable.csv', parse_dates= True)

df_r.head()

,country_or_area,ISO,commodity_transaction,year,category,new_unit,new_quantity,renewable,non_renewable,difference
0,Afghanistan,AFG,Electricity - Gross production,1994,total_electricity,"Kilowatt-hours, thousand",687000.00,472000.00,215000.00,0.00
1,Afghanistan,AFG,Electricity - Gross production,1995,total_electricity,"Kilowatt-hours, thousand",675000.00,466000.00,209000.00,0.00
2,Afghanistan,AFG,Electricity - Gross production,1996,total_electricity,"Kilowatt-hours, thousand",675000.00,475000.00,200000.00,0.00
3,Afghanistan,AFG,Electricity - Gross production,1997,total_electricity,"Kilowatt-hours, thousand",670000.00,485000.00,185000.00,0.00
4,Afghanistan,AFG,Electricity - Gross production,1998,total_electricity,"Kilowatt-hours, thousand",665000.00,495000.00,170000.00,0.00


In [4]:
total_electricity=df_f.loc[(df_f['commodity_transaction'] == 'Electricity - Gross production')]
total_electricity.category.unique()


array(['total_electricity'], dtype=object)

In [5]:
total_at_world = total_electricity.groupby('year')['new_quantity'].sum().sort_values(ascending=True).reset_index()
total_at_world


,year,new_quantity
0,1994,12886.11
1,1995,13324.75
2,1996,13753.31
3,1997,14042.59
4,1998,14370.60
5,1999,14801.23
6,2000,15496.09
7,2001,15597.99
8,2002,16215.15
9,2003,16812.14


In [6]:
output_notebook()
p = figure()
p.line(x='year', y='new_quantity', line_width=2, source = total_at_world)
p.yaxis.formatter=NumeralTickFormatter(format="00")
p.xaxis.axis_label = 'Rok'
p.yaxis.axis_label = 'Zużycie kV/h'
show(p)

Loading BokehJS ...

In [7]:
def _get_country_per_year_stats(
    data_renewable: pd.DataFrame,
    country: str, 
    year: int
) -> pd.DataFrame:
    """ Gets stats of country per year"""

    yearlist = [year-2,year-1,year]
    frame_for_year = data_renewable.loc[(data_renewable['year'] == year)].sort_values(by= 'new_quantity', ascending=True).reset_index(drop=True)
    contry_frame_3years = data_renewable[ (data_renewable['year'].isin(yearlist)) & (data_renewable['country_or_area'] == country)]
    country_frame = data_renewable[(data_renewable['year'] == year) & (data_renewable['country_or_area'] == country)]
    top3 = frame_for_year.head(3)
    bot3 = frame_for_year.tail(3)
    frameList = [top3,country_frame,bot3]
    ranking = pd.concat(frameList)
    rankinglist = [1,2,3,frame_for_year[frame_for_year['country_or_area'] == country].index[0]+1,bot3.index[0]+1,bot3.index[1]+1,bot3.index[2]+1]
    ranking.insert(loc=0, column= 'Ranking', value = rankinglist )
    
    return ranking

In [8]:
_get_country_per_year_stats(df_r,'Poland',2014)

,Ranking,country_or_area,ISO,commodity_transaction,year,category,new_unit,new_quantity,renewable,non_renewable,difference
0,1,Niue,NIU,Electricity - Gross production,2014,total_electricity,"Kilowatt-hours, thousand",3226.67,66.67,3160.00,0.00
1,2,Tuvalu,TUV,Electricity - Gross production,2014,total_electricity,"Kilowatt-hours, thousand",5100.00,0.00,5100.00,0.00
2,3,St. Helena and Depend.,SHN,Electricity - Gross production,2014,total_electricity,"Kilowatt-hours, thousand",10670.00,1080.00,9590.00,0.00
3380,205,Poland,POL,Electricity - Gross production,2014,total_electricity,"Kilowatt-hours, thousand",159059000.00,10417000.00,148642000.00,0.00
226,227,India,IND,Electricity - Gross production,2014,total_electricity,"Kilowatt-hours, thousand",1308873000.00,197532000.00,1111341000.00,0.00
227,228,United States,USA,Electricity - Gross production,2014,total_electricity,"Kilowatt-hours, thousand",4339210000.00,1339316000.00,2999894000.00,0.00
228,229,China,CHN,Electricity - Gross production,2014,total_electricity,"Kilowatt-hours, thousand",5649583500.00,1368142000.00,4281441500.00,0.00


In [9]:
def _get_country_yearly_stats(
    data: pd.DataFrame,
    country: str, 
) -> pd.DataFrame:
    """ Gets stats of selected country throught years
        Working at final_version.csv file 
    """

    import seaborn as sns
    from bokeh.layouts import gridplot
    from bokeh.io import output_notebook, show, reset_output
    from bokeh.plotting import figure
    from bokeh.models import HoverTool, ColumnDataSource, CDSView, BooleanFilter, Legend
    from bokeh.palettes import Spectral
    from bokeh.models import NumeralTickFormatter
    
    source_list = ['Electricity - total nuclear production',
                'Electricity - total geothermal production',
                'Electricity - total tide, wave and hydro production',
                'Electricity - total solar production',
                'Electricity - total wind production','Electricity - total thermal production','Electricity - Gross production']

    df_ele = data.loc[(data.commodity_transaction.isin(source_list))]

    source_list_short = {'Electricity - total nuclear production': 'Nuclear Production',
                'Electricity - total geothermal production': 'Geothermal Production',
                'Electricity - total tide, wave and hydro production': 'Tide, Wave and Hydro Production',
                'Electricity - total solar production': 'Solar Production',
                'Electricity - total wind production': 'Wind Production',
                'Electricity - total thermal production': 'Thermal Production',
                'Electricity - Gross production': 'Gross Production'}

    df_ele ['commodity_transaction'] = df_ele.commodity_transaction.map(source_list_short)         
          
    output_notebook()

    country_for_pivot = df_ele.loc[(df_ele['country_or_area'] == country)]

    names = list(country_for_pivot['commodity_transaction'].unique())
    names.remove('Gross Production')
    Reverse_Spectral= Spectral.copy()
    Reverse_Spectral[len(names)]

    total_by_source = pd.pivot_table(country_for_pivot, values = 'new_quantity', index=['year'], columns =['commodity_transaction'], fill_value=0)
    
    
    for i in names:
        total_by_source[i + ' %'] = (total_by_source[i])*100/(total_by_source[names].sum(axis=1))

    
    cm = sns.light_palette("blue", as_cmap=True)
    a = total_by_source.style.background_gradient(cmap=cm)

    
    
    source = source = ColumnDataSource(total_by_source)
    
    tooltips_source = [
        ('year', '@year'),
        ('By Source', '@$name{0,0} KWh. M'),
        ('Total','$y{0,0} KWh. M')
    ]

    subplot1 = figure(title = 'Electricity Production in {} by Source 1994-2014'.format(country),plot_width=800, plot_height=600,)
    subplot1.varea_stack(names,
                    x='year',color=Reverse_Spectral[len(names)] , legend_label = names, source = source)

    subplot1.vline_stack(names,
                    x='year',color = 'black', source = source)
    
    subplot1.line(x = 'year', y = 'Gross Production',
                line_dash=[4, 4], line_color='gray', line_width=2, legend_label='Gross Production', source = source)
    subplot1.add_layout(subplot1.legend[0], 'right')
    subplot1.legend.location = "bottom_right"
    #subplot1.legend.location ='outside'
    subplot1.add_tools(HoverTool(tooltips = tooltips_source))
    subplot1.yaxis.formatter=NumeralTickFormatter(format="00")
    show(subplot1)
    
    return a 
   

In [10]:
_get_country_yearly_stats(df_f,'Poland') #działa na df_final

C:\Users\karl1\AppData\Local\Temp\ipykernel_26548\3122336756.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ele ['commodity_transaction'] = df_ele.commodity_transaction.map(source_list_short)


Loading BokehJS ...

commodity_transaction,Gross Production,Solar Production,Thermal Production,"Tide, Wave and Hydro Production",Wind Production,Solar Production %,Thermal Production %,"Tide, Wave and Hydro Production %",Wind Production %
year,,,,,,,,,
1994,135.347000,0.000000,131.561000,3.786000,0.000000,0.000000,97.202746,2.797254,0.000000
1995,139.006000,0.000000,135.154000,3.851000,0.001000,0.000000,97.228897,2.770384,0.000719
1996,143.173000,0.000000,139.263000,3.910000,0.000000,0.000000,97.269038,2.730962,0.000000
1997,142.790000,0.000000,138.972000,3.816000,0.002000,0.000000,97.326143,2.672456,0.001401
1998,142.789000,0.000000,138.458000,4.327000,0.004000,0.000000,96.966853,3.030345,0.002801
1999,142.128000,0.000000,137.842000,4.282000,0.004000,0.000000,96.984408,3.012777,0.002814
2000,145.184000,0.000000,141.063000,4.116000,0.005000,0.000000,97.161533,2.835023,0.003444
2001,145.616000,0.000000,141.382000,4.220000,0.014000,0.000000,97.092352,2.898033,0.009614
2002,144.126000,0.000000,140.159000,3.906000,0.061000,0.000000,97.247547,2.710129,0.042324


In [11]:
def _map_for_world(
    data: pd.DataFrame,
) -> pd.DataFrame:
    """ Print map for world with electic stats 
        Working at final_version.csv file 
    """
    import plotly.express as px 
    df_m = data.loc[(data['commodity_transaction']=='Electricity - Gross production')] 
    df_m.rename({'new_quantity':'Electric_Energy_Production'},axis=1, inplace=True)
    df_m=df_m.sort_values("year")
    
    map = px.choropleth(df_m,             
              locations="ISO",               
              color=np.log10(df_m['Electric_Energy_Production']),
              hover_name="country_or_area",  
              animation_frame="year",
              color_continuous_scale="Blues",
              hover_data=['Electric_Energy_Production'],
              range_color=tuple(np.log2([1, 9])),
              height=700  
             )
    map.show()


In [12]:
_map_for_world(df_f)

C:\Users\karl1\AppData\Local\Temp\ipykernel_26548\2215934235.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m.rename({'new_quantity':'Electric_Energy_Production'},axis=1, inplace=True)


In [13]:
def _production_structure_for_world(
    data: pd.DataFrame,
    ) -> pd.DataFrame:
    """ Return production structure for world 
        Working at final_version.csv file 
    """
    import seaborn as sns
    from bokeh.layouts import gridplot
    from bokeh.io import output_notebook, show, reset_output
    from bokeh.plotting import figure
    from bokeh.models import HoverTool, ColumnDataSource, CDSView, BooleanFilter, Legend
    from bokeh.palettes import Spectral
    


    source_list = ['Electricity - Gross production',
       'Electricity - total thermal production',
       'Electricity - total tide, wave and hydro production',
       'Electricity - total solar production',
       'Electricity - total nuclear production',
       'Electricity - total wind production',
       'Electricity - total geothermal production']
    df_fp = data.loc[(data.commodity_transaction.isin(source_list))]
    total_by_source = pd.pivot_table(df_fp , values = 'new_quantity', index=['year'], columns =['commodity_transaction'], aggfunc = ('sum'), fill_value=0).reset_index()
    
    names = ['Electricity - total thermal production',
       'Electricity - total tide, wave and hydro production',
       'Electricity - total solar production',
       'Electricity - total nuclear production',
       'Electricity - total wind production',
       'Electricity - total geothermal production']
    
    for i in names:
        total_by_source[i + ' %'] = (total_by_source[i])*100/(total_by_source[names].sum(axis=1))
    
    source = source = ColumnDataSource(total_by_source)
    Reverse_Spectral= Spectral.copy()

    tooltips_source = [
    ('year', '@year'),
    ('By Source', '@$name{0,0} TWh'),
    ('Total','$y{0,0} TWh')
    ]
    output_notebook()
    subplot = figure(title = 'Electricity Production by Source 1994-2014',plot_width=900, plot_height=500)
    subplot.varea_stack(names,
                x='year',color=Reverse_Spectral[6] , legend_label = names, source = source)

    subplot.vline_stack(names,
                x='year',color = 'black', source = source)
    subplot.line(x = 'year', y = 'Electricity - Gross production',
            line_dash=[4, 4], line_color='gray', line_width=2, legend_label='Gross', source = source)

    subplot.legend.location ='top_left'
    subplot.add_tools(HoverTool(tooltips = tooltips_source))
    subplot.add_layout(subplot.legend[0], 'right')
    show(subplot)
        

In [14]:
_production_structure_for_world(df_f)

Loading BokehJS ...

In [15]:
def _perc_stats_for_world(
       data: pd.DataFrame,
       ) -> pd.DataFrame:
       """ Return percentage share of each sector in production for the world 
        Working at final_version.csv file 
        """
       import seaborn as sns
       from bokeh.layouts import gridplot
       from bokeh.io import output_notebook, show, reset_output
       from bokeh.plotting import figure
       from bokeh.models import HoverTool, ColumnDataSource, CDSView, BooleanFilter, Legend
       from bokeh.palettes import Spectral
       Reverse_Spectral= Spectral.copy()

       source_list = ['Electricity - Gross production',
              'Electricity - total thermal production',
              'Electricity - total tide, wave and hydro production',
              'Electricity - total solar production',
              'Electricity - total nuclear production',
              'Electricity - total wind production',
              'Electricity - total geothermal production']
       df_fp = data.loc[(data.commodity_transaction.isin(source_list))]

       total_by_source_perc = pd.pivot_table(df_fp , values = 'new_quantity', index=['year'], columns =['commodity_transaction'], aggfunc = ('sum'), fill_value=0).reset_index()

       names = ['Electricity - total thermal production',
              'Electricity - total tide, wave and hydro production',
              'Electricity - total solar production',
              'Electricity - total nuclear production',
              'Electricity - total wind production',
              'Electricity - total geothermal production']
       nm_gross = ['Gross']

       for i in names:
              total_by_source_perc[i + ' %'] = (total_by_source_perc[i])*100/(total_by_source_perc[names].sum(axis=1))
       
       total_by_source_perc = total_by_source_perc.drop(['Electricity - total thermal production',
              'Electricity - total tide, wave and hydro production',
              'Electricity - total solar production',
              'Electricity - total nuclear production',
              'Electricity - total wind production',
              'Electricity - total geothermal production'], axis=1)  

       cm = sns.light_palette("blue", as_cmap=True)
       a = total_by_source_perc.style.background_gradient(cmap=cm)
       
       percentage_names = ['Electricity - total thermal production %',
                    'Electricity - total tide, wave and hydro production %',
                    'Electricity - total solar production %',
                    'Electricity - total nuclear production %',
                    'Electricity - total wind production %',
                    'Electricity - total geothermal production %']
       
       output_notebook()
       source_perc = ColumnDataSource(total_by_source_perc)
       subplot3 = figure( title = 'Percentage share of each sector in production for the World',plot_width=900, plot_height=600)
       subplot3.vbar_stack(percentage_names,x='year', width=0.5, line_color='black',color= Reverse_Spectral[6],legend_label = percentage_names, source =source_perc)
       
       tooltips_mix = [
       ('year', '@year'),
       ('By Source', '@$name{0.00} %'),
       ('source','$name')
       ]
       subplot3.add_layout(subplot3.legend[0], 'right')
       subplot3.add_tools(HoverTool(tooltips = tooltips_mix))
       show(subplot3)
       
       return a

In [16]:
_perc_stats_for_world(df_f)

Loading BokehJS ...

commodity_transaction,year,Electricity - Gross production,Electricity - total thermal production %,"Electricity - total tide, wave and hydro production %",Electricity - total solar production %,Electricity - total nuclear production %,Electricity - total wind production %,Electricity - total geothermal production %
0,1994,12886.108253,63.392133,18.819519,0.007163,17.409457,0.056221,0.315507
1,1995,13324.751389,62.991258,19.138195,0.007288,17.504503,0.058949,0.299808
2,1996,13753.309573,63.208897,18.830910,0.007831,17.577121,0.067821,0.307420
3,1997,14042.592982,63.889427,18.672518,0.008359,17.042867,0.083953,0.302875
4,1998,14370.600973,64.239604,18.310986,0.008120,17.017374,0.109211,0.314705
5,1999,14801.231135,64.564715,17.854132,0.007280,17.101401,0.144344,0.328128
6,2000,15496.088666,65.291325,17.471008,0.009512,16.709148,0.198440,0.320566
7,2001,15597.990241,65.588611,16.920147,0.011323,16.910170,0.239806,0.329943
8,2002,16215.148670,66.213075,16.718968,0.013167,16.413915,0.318939,0.321937
9,2003,16812.144850,67.396505,16.211027,0.015203,15.682770,0.374356,0.320140
